In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import random

# Load data

In [ ]:
data_dir = '../input/celeba-dataset/'
df_partition = pd.read_csv(f'{data_dir}list_eval_partition.csv')
img_dir = f'{data_dir}img_align_celeba/img_align_celeba/'
train_paths = [img_dir + i for i in df_partition[df_partition['partition'] == 0]['image_id']]
val_paths = [img_dir + i for i in df_partition[df_partition['partition'] == 1]['image_id']]
test_paths = [img_dir + i for i in df_partition[df_partition['partition'] == 2]['image_id']]
print(f'Train: {len(train_paths)}')
print(f'Val: {len(val_paths)}')
print(f'Test: {len(test_paths)}')

In [ ]:
def load_image(path, max_dim=None, gray=False):
    img = Image.open(path)
    if max_dim:
        img.thumbnail((max_dim, max_dim))
    if gray:
        img = img.convert('L')
    return np.array(img)


def normalize(img):
    img = img / 255
    return img


def denormalize(img):
    img = img * 255
    return img


def show(img, gray=False):
    plt.axis('off')
    if gray:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img)

In [ ]:
max_dim = 88
gray = False

In [ ]:
im = load_image(train_paths[100000], max_dim=max_dim, gray=gray)
image_shape = im.shape
print(f'Shape: {image_shape}')
show(im, gray)

In [ ]:
train_images = np.array([load_image(path, max_dim=max_dim, gray=gray)
                         for path in train_paths[: 2 ** 14]])
train_images = normalize(train_images)
train_images.shape

In [ ]:
test_images = np.array([load_image(path, max_dim=max_dim, gray=gray)
                         for path in test_paths[: 2 ** 11]])
test_images = normalize(test_images)
test_images.shape

# The Autoencoder

In [ ]:
class Autoencoder(Model):
    def __init__(self, encoded_dim):
        super(Autoencoder, self).__init__()
        self.encoded_dim = encoded_dim   
        self.encoder = tf.keras.Sequential([
            layers.Flatten(),
            layers.Dense(encoded_dim, activation='relu')
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(np.prod(image_shape), activation='sigmoid'),
            layers.Reshape(image_shape)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Change the parameter to change the number of encoded dimensions
# autoencoder = Autoencoder(64)
autoencoder = Autoencoder(np.prod(image_shape) // 8)

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
autoencoder.fit(train_images, train_images,
                epochs=10,
                shuffle=True,
                validation_data=(test_images, test_images))

In [ ]:
encoded_imgs = autoencoder.encoder(test_images).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
n = 10
rand = random.randint(0, len(test_images) - n)

plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_images[i + rand])
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i + rand])
    plt.title('reconstructed')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()